In [1]:
import torch
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from PIL import Image
import  os, sys
from tqdm import tqdm
sys.path.append("..")
sys.path.append("../..")
sys.path.append("../../..")

In [2]:
sys.path.append("..")
sys.path.append("../..")
sys.path.append("src/")
sys.path.append("src/OpenFoam")
sys.path.append("src/diffusion_notebooks")
sys.path.append("data/")
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
from diffusion_core.diffusion import GaussianDiffusion1D
from diffusion_core.model import Unet1D
from pathlib import Path

/home/bardiya/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/src/optimization_loop/Inner_loop/../../diffusion_core/diffusion.py:331: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


# Classes

In [3]:
from models import UA_surrogate_model

if __name__  == "__main__":
    model = UA_surrogate_model()
    x = torch.zeros((2,384))
    out = model(x)
    print(out)


[tensor([[-0.2752, -0.0072],
        [-0.2752, -0.0072]], grad_fn=<CatBackward0>), tensor([[-0.1909, -0.0035],
        [-0.1909, -0.0035]], grad_fn=<CatBackward0>), tensor([[0.0310, 1.6393],
        [0.0310, 1.6393]], grad_fn=<CatBackward0>), tensor([[-0.2294,  0.0017],
        [-0.2294,  0.0017]], grad_fn=<CatBackward0>)]


In [4]:

device = "cpu"
# device = "cuda"
# Same architecture as in training
model = Unet1D(
    dim=32,
    dim_mults=(2, 4, 8, 16),
    channels=2,  # X and Y
    dropout=0.1
).to(device)  # or .to(device)

# Create the same diffusion wrapper
diffusion = GaussianDiffusion1D(
    model,
    seq_length=192,      # must match your training setup
    objective='pred_noise',
    timesteps=1000,
    auto_normalize=False
).to(device)  # or .to(device)

In [5]:
import scipy.io as sio
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from utils import  BO_surrogate_uncertainty
sys.path.append("../../OpenFoam")
from OpenFoam.Airfoil_simulation_1.ShapeToPerformance import shape_to_performance as STP1

DATA_DIR = Path(rf"../../../data")
coord_mm = np.load(DATA_DIR/"coord_min_max.npy")  # [[x_min,y_min],[x_max,y_max]]
x_min,y_min = coord_mm[0]; x_max,y_max = coord_mm[1]

def inv_coords(xs_s):                   # xs_s shape (...,2,192) tensor
    xs_np = xs_s.permute(0,2,1).cpu().numpy()    # -> (B,192,2)
    xs_np[...,0] = xs_np[...,0]*(x_max-x_min) + x_min
    xs_np[...,1] = xs_np[...,1]*(y_max-y_min) + y_min
    return xs_np                                # (B,192,2) numpy

def init_generate_samples_latents(model ,diffusion, checkpoint_path, NUM_TO_GENERATE , BATCH_SIZE):
    """
        input:
            checkpoint_path:    the loading path for the weigths of the
                                diffusion unet.
            NUM_TO_GENERATE
            BATCH_SIZE
        output: 
            all_latents:    list of the latents used for sampling phase
            all_shapes:     list of the shapes  generated from the latents
    """
    # Load checkpoint
    # checkpoint_path = rf"src/diffusion_notebooks/DIffusion_model_weigths_and_datas/dpp_0.1_autonorm_true_125_from_base_ddpm/model_epoch_124.pt"
    model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
    model.eval()
    print("Loaded model weights from:", checkpoint_path)

    num_to_generate = NUM_TO_GENERATE
    batch_size      = BATCH_SIZE

    all_latent = []
    all_shapes = []
    with torch.no_grad():
        done = 0
        while done < num_to_generate:
            cur = min(batch_size, num_to_generate - done)
            
            latent = torch.randn((cur,2,192)).to(device)
            samples = diffusion.latent_sample(latent , is_ddim=True)
            generated_real = inv_coords(samples)
            
            all_latent.append(latent.cpu().detach().numpy())
            all_shapes.append(generated_real)
            done += cur
            print(f"Generated {done}/{num_to_generate}")

    np.save(os.path.join("Database" , "DB_innerloop.npy") , {
        "latents": np.vstack(all_latent),
        "shapes": np.vstack(all_shapes),
    })

def GEN_UA(number_iter = 0,number_generations=100 , population_size = 1000 , from_DB_innerloop = True):
    # n_iter = 2
    print('calculating surrogate pareto ...')
    if from_DB_innerloop:
        DB_innerloop = np.load(os.path.join("Database" , "DB_innerloop.npy"),allow_pickle=True).item()
        full_sampels = DB_innerloop["latents"].reshape(DB_innerloop["latents"].shape[0], -1)  # shape: (batch, 384)
        print(full_sampels.shape)
    problem_uncertainty = BO_surrogate_uncertainty(n_iter=number_iter)
    algorithm = NSGA2(pop_size=population_size)
    res = minimize(problem_uncertainty,
                algorithm,
                ('n_gen', number_generations),
                seed=1,
                verbose=False,
                X = full_sampels  if from_DB_innerloop else None)
    Paretoset_uncertainty = res.X
    Out_surrogate_uncertainty = res.F
    # sio.savemat('surrogate_pareto/ParetoSet_test.mat' , {'ParetoSet': np.array(Paretoset_uncertainty)})
    # sio.savemat('surrogate_pareto/Out_surrogate_test.mat', {'Out_surrogate': np.array(Out_surrogate_uncertainty)})
    np.save(os.path.join("Database" , "DB_NSGA.npy"),{
        'ParetoSet': np.array(Paretoset_uncertainty), # the latents
        'Out_surrogate': np.array(Out_surrogate_uncertainty)
    })

def NSGA_latent_to_shape(model ,diffusion, docker_mount_path, checkpoint_path, BATCH_SIZE=128):

    DB_NSGA = np.load(os.path.join("Database" , "DB_NSGA.npy"),allow_pickle=True).item()

    # reset the latent to (batch,2,192)
    NSGA_latent = DB_NSGA["ParetoSet"].reshape(DB_NSGA["ParetoSet"].shape[0],2,-1)
    NSGA_latent = torch.from_numpy(NSGA_latent).to(torch.float32)

    model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
    model.eval()
    print("Loaded model weights from:", checkpoint_path)

    num_to_generate = len(NSGA_latent)
    batch_size = BATCH_SIZE

    all_latent = []
    all_shapes = []
    with torch.no_grad():
        done = 0
        while done < num_to_generate:
            cur = min(batch_size, num_to_generate - done)
            
            latent = NSGA_latent[done:done+cur,:,:].to(device)
            samples = diffusion.latent_sample(latent , is_ddim=True)
            generated_real = inv_coords(samples)
            
            all_latent.append(latent.cpu().detach().numpy())
            all_shapes.append(generated_real)
            done += cur
            print(f"Generated {done}/{num_to_generate}")

    np.save(os.path.join(docker_mount_path , "DB_CFD.npy") , {
        "latents": np.vstack(all_latent),
        "shapes": np.vstack(all_shapes),
        "performance":[]
    })

def CFD_simulation(docker_container_id):
    # command = fr"docker exec {docker_container_id} python3 /home/airfoil_UANA/performance_finding.py"
    command = (
        f'docker exec {docker_container_id} bash -c "'
        f'source /opt/openfoam5/etc/bashrc && '
        f'cd /home/airfoil_UANA && '
        f'python3 innerloop_performance_finding.py"'
    )
    os.system(command)



In [6]:
"""
    The main Hyperparams
"""
docker_mount_path = "../../OpenFoam"
NUM_TO_GENERATE = 2
BATCH_SIZE = 2
docker_container_id = "f897792b6b56" 
saving_path = rf"src/optimization_loop/Inner_loop/Database/DB_innerloop.npy"
Unet_checkpoint_path = rf"../../../src/diffusion_notebooks/DIffusion_model_weigths_and_datas/dpp_0.1_autonorm_true_125_from_base_ddpm/model_epoch_124.pt"
"""
******************************
******************************
warning:
    before running make sure to copy src/OpenFoam/Airfoil_simulation_1/OpenFOAM_0
    in that folder 200 times with new directories name as src/OpenFoam/Airfoil_simulation_1/OpenFOAM_i for the i'th core ussage
******************************
******************************
"""



"\n******************************\n******************************\nwarning:\n    before running make sure to copy src/OpenFoam/Airfoil_simulation_1/OpenFOAM_0\n    in that folder 200 times with new directories name as src/OpenFoam/Airfoil_simulation_1/OpenFOAM_i for the i'th core ussage\n******************************\n******************************\n"

# Main Procedure
1.  creating and generating the `DB_innerloop` for the shapes and latents holding (just these two) ->  OK
2.  making the `UA_information` data for the whole `DB_innerloop` and save it in `DB_UA` for the NSGA algorithm ->  OK
3.  using the `NSGA` algorithm and  creating the `DB_NSGA` -> OK
4.  give the results of the `DB_NSGA` to the Openfoam and  create `DB_OpenFoam`
5.  evaluate and tag the `DB_Openfoam` and append the correct data to the `DB_Valid` and `DB_Invalid`
6.  retrain the `UA_surrogate_Model` with the valid datas
7.  redo  all above part for `N` Epoch 

In [7]:
# # Stage 1:
# init_generate_samples_latents(model , 
#                             diffusion , 
#                             NUM_TO_GENERATE=NUM_TO_GENERATE,
#                             BATCH_SIZE=BATCH_SIZE,
#                             checkpoint_path=Unet_checkpoint_path
#                             )

In [8]:
import scipy.io as sio
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from utils import  BO_surrogate_uncertainty

# Stage 2:
GEN_UA(number_generations=1, population_size=5,from_DB_innerloop=True)


calculating surrogate pareto ...
(2, 384)

Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False



In [9]:
# Stage 3:
NSGA_latent_to_shape(model , 
                diffusion , 
                BATCH_SIZE=BATCH_SIZE,
                checkpoint_path=Unet_checkpoint_path,
                docker_mount_path=docker_mount_path
                )


Loaded model weights from: ../../../src/diffusion_notebooks/DIffusion_model_weigths_and_datas/dpp_0.1_autonorm_true_125_from_base_ddpm/model_epoch_124.pt


sampling loop time step: 100%|██████████| 1000/1000 [00:26<00:00, 37.48it/s]


Generated 2/3


sampling loop time step: 100%|██████████| 1000/1000 [00:14<00:00, 71.18it/s]

Generated 3/3


In [ ]:
# Stage 4 (first  start it):
CFD_simulation(docker_container_id=docker_container_id)

In [10]:
hold = np.load(os.path.join( docker_mount_path, "DB_CFD.npy"),allow_pickle=True).item()
a = hold["performance"]
len(a)

3

In [11]:
a[0]

array([-1000.,  1000.,     0.])